In [1]:
from account import * # keys and tokkens
import tweepy
from textblob import TextBlob
import pandas as pd 
import re


In [2]:
# setup tweepy 

auth = tweepy.OAuthHandler(ConsumerKey, ConsumerSecret) 
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth) 

In [3]:
#get the top trends in israel


locations = api.trends_available()
df =pd.DataFrame(locations)
israel = df.loc[df['country'] == 'Israel']
woeid = list(israel['woeid'])


def get_trend(x=10):

    df = pd.DataFrame()

    for  i in woeid:
        trends = api.trends_place(i)[0]
        trends = trends['trends']
        A = pd.DataFrame(trends)
        df = df.append(A ,ignore_index=True)



    return df.sort_values('tweet_volume', ascending=False).drop_duplicates().reset_index(drop=True).head(x)


get_trend(15)

,name,promoted_content,query,tweet_volume,url
0,Russia,None,Russia,938233.0,http://twitter.com/search?q=Russia
1,Russia,None,Russia,936463.0,http://twitter.com/search?q=Russia
2,World Cup,None,%22World+Cup%22,646760.0,http://twitter.com/search?q=%22World+Cup%22
3,World Cup,None,%22World+Cup%22,644732.0,http://twitter.com/search?q=%22World+Cup%22
4,Argentina,None,Argentina,604751.0,http://twitter.com/search?q=Argentina
5,Argentina,None,Argentina,603125.0,http://twitter.com/search?q=Argentina
6,Spain,None,Spain,585206.0,http://twitter.com/search?q=Spain
7,Spain,None,Spain,584691.0,http://twitter.com/search?q=Spain
8,#WalkAway,None,%23WalkAway,349649.0,http://twitter.com/search?q=%23WalkAway
9,#WalkAway,None,%23WalkAway,348032.0,http://twitter.com/search?q=%23WalkAway


In [4]:
#clean tweets


def clean_tweet(sentence):
    sentence = re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+','', sentence)
    sentence = re.sub(r'\@\w+ *'," ",sentence)
    return sentence


def is_RT (tweet):
    RT = re.findall(r'RT',tweet)
    if  RT :
        return True  
    else :
        return False

In [5]:
#transform html file to list of twitters 
# html file with info comes from the KnessetMembersTwitterAccounts1.0 scraper i made using selenium 

df = pd.read_html('KnessetMembersTwitter.html')
Xks_info = df[0]
accounts = Xks_info[Xks_info['twitter'] != 'none']
accounts = list(accounts['twitter'])


#change twitter urls to format @ABCD

for i in range(len(accounts)) :
   
    accounts[i] = '@'+ accounts[i].replace('https://twitter.com/','').replace('?lang=he','').replace('http://twitter.com/#!/','').replace('http://twitter.com/','').replace('ttps://twitter.com/','')  

In [6]:
# array of tweets for each account in list 

def get_tweets(screenName,num_items):


    all_accounts = []
    

    for i in screenName[0:10] :  #only first 10 members for development
        accounts_tweets = []
        for tweet in tweepy.Cursor(api.user_timeline,screen_name = i).items(num_items):
            #text= tweet.text
            text= clean_tweet(tweet.text)
            #print(tweet.favorite_count)
            #print(tweet.in_reply_to_user_id_str)
            #print(tweet.author.screen_name)
            #print('----')
            accounts_tweets.append(text)
            
        all_accounts.append(accounts_tweets)

    return all_accounts


screenName = accounts # the accounts to retrieve tweets from 
num_items = 2 # num of tweets for each account 
tweets= get_tweets(accounts, num_items)
tweets

[['לָשׂוּם לַאֲבֵלֵי צִיּוֹן לָתֵת לָהֶם פְּאֵר תַּחַת אֵפֶר שֶׁמֶן שָׂשׂוֹן תַּחַת אֵבֶל\n\nידיעה מרגשת על הבוקר! לנתן… ',
  'בועדת הכלכלה, פותחים את יום הפיקוח הפרלמנטרי על תכניות העבודה של הממשלה!\nתפקיד הכנסת הוא לפקח על הממשלה, ועלינו לעש… '],
 ['RT  : Australia has terminated our annual $10m funding to Palestinian Authority, redirecting funds to  Humanitarian Fund to…',
  '  הנקודה היא (כפי שהסברתי בהמשך) שהאחריות הבלעדית למצבם העגום של הילדים מונח לפתחה של הרש"פ (או… '],
 ['J Street’s problem is not with Amb. Friedman but with democracy. It supported the previous ambassador who represent… ',
  'הארץ או ארץ נהדרת? עיתון הארץ קורא למינויו של ח״כ איימן עודה, יו״ר הרשימה המשותפת לראש האופוזיציה בהיותו כ״בעד שלום… '],
 ['&gt;&gt;להסתכל לציבור הישראלי בעיניים ולהסביר לו מדוע בסופו של תהליך, שמלווה בהרבה ברכות וחיוכים, כספי משכורות המחבלים יג… ',
  'הסעיף השערורייתי שאושר היום בדיון על קיזוז משכורות המחבלים, אשר הייתי היחיד שהתנגד לו, אומר שהכסף לא באמת יקוזז מהר… '],
 ['כך נראה איבוד 

In [7]:
##### input tweets array with each members array from get_tweets
##### output word count for each member

def WWWords (tweets) :   
    m= []
    for i in range(len(tweets)) :
        
        m.append( word_count(listOfTweetsToWords(tweets[i])))
        
    return countsArray


#wordCountsArray = WWWords(tweets)

In [ ]:
def listOfTweetsToWords(tweetList):

    word_list = ''.join(tweetList)
    word_list = word_list.split()
    return word_list


def word_count(tweets):
    
    if type(tweets) is str :
        word_list = tweets.split()      
    else:
        word_list = tweets

    word_count={}

    for word in word_list:
        word_count[word] = word_count.get(word,0) +1
    return word_count

In [8]:
#sentiment

def sentiment(word, lang = 'en'):

    twitts = api.search(word,lang)
    for twitt in twitts : 
        print(twitt.text)
        analysis = TextBlob(twitt.text)
        print(analysis.sentiment)
        print(twitt.author.screen_name)
        print("")

    
#sentiment('Israel')

In [9]:
def getreaction (searchTerm,noOfSearches):

    def precentage (part,whole):
        return 100 * float(part)/float(whole)

    polarity = 0
    negative = 0
    positive = 0
    neutral = 0

    tweets = tweepy.Cursor(api.search ,q= searchTerm).items(noOfSearches)

    for tweet in tweets :
        analysis = TextBlob(tweet.text)
        polarity += analysis.sentiment.polarity

        if (analysis.sentiment.polarity == 0):
            neutral += 1
        elif (analysis.sentiment.polarity > 0):
            positive += 1
        elif (analysis.sentiment.polarity < 0):
            negative += 1
        
    positive = format(precentage(positive, noOfSearches) ,'.0f')
    negative = format(precentage(negative, noOfSearches) ,'.0f')
    neutral = format(precentage(neutral, noOfSearches)  ,'.0f')    

    if (polarity == 0):
           reaction = 'neutral' 
    elif (polarity > 0):
           reaction = 'POSITIVE'
    elif (polarity < 0):
            reaction = 'NEGATIVE' 


    print ('People reaction to {}  according to {} tweets is {}'.format(searchTerm,noOfSearches,reaction))

    print()
    print(str(positive) + "% people thought it was positive")
    print(str(negative) + "% people thought it was negative")
    print(str(neutral) + "% people thought it was neutral")

    
word = 'israel'
num = 100

#getreaction(word,num)